In [ ]:
query = "mar thoma church english holy communion"
query2 = "mar thoma church english holy qurbana"

In [215]:
import googleapiclient

In [216]:
print(googleapiclient)

AttributeError: module 'googleapiclient' has no attribute '__version__'

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
API_KEY = os.environ.get("API_KEY")

In [51]:
import googleapiclient.discovery
import googleapiclient.errors

import pytz
from datetime import datetime, timedelta


In [3]:
youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=API_KEY)

In [203]:
def convert_utc_to_est(utc_timestamp):
    # parse the UTC timestamp into a datetime object
    utc_time = datetime.strptime(utc_timestamp, '%Y-%m-%dT%H:%M:%SZ')
    
    # create timezone objects for UTC and EST
    utc_zone = pytz.utc
    est_zone = pytz.timezone('US/Eastern')
    
    # convert the UTC datetime to EST datetime
    est_time = utc_zone.localize(utc_time).astimezone(est_zone)

    formatted_est_time = est_time.strftime('%A, %B %d, %Y: %I:%M %p EST')
    
    # format the EST datetime as a string in ISO format
    return est_time, formatted_est_time

def get_recent_weekend(est_time):
    # get the current time in EST timezone
    est_zone = pytz.timezone('US/Eastern')
    
    # calculate the start of the most recent weekend
    if est_time.weekday() == 5:  # it's currently Saturday
        start_time = datetime(est_time.year, est_time.month, est_time.day, 0, 0, 0, 0, est_zone)
    else:
        days_since_saturday = (est_time.weekday() + 1) % 7  # 0 = Saturday, 1 = Sunday, etc.
        start_time = est_time - timedelta(days=days_since_saturday+1, hours=est_time.hour, minutes=est_time.minute, seconds=est_time.second, microseconds=est_time.microsecond)
    
    # calculate the end of the most recent weekend
    end_time = start_time + timedelta(days=1, hours=23, minutes=59, seconds=59)
    
    # format the start and end times as strings
    start_time_str = start_time.strftime('%A, %B %d, %Y: %I:%M %p EST')
    end_time_str = end_time.strftime('%A, %B %d, %Y: %I:%M %p EST')
    print(start_time_str, end_time_str)
    
    return start_time, end_time


# get the latest videos from the most recent Saturday and Sunday
# Ideally, we are updating multiple times on Sunday morning

def get_youtube_videos(query: str):
    request = youtube.search().list(
        part="snippet",
        maxResults=25,
        order = "date",
        q=query
    )
    response = request.execute()

    # filter videos based on time; we want videos to fall in most recent weekend
    # we make exceptions for live videos and upcoming livestreams
    start_date, end_date = get_recent_weekend(est_time=datetime.now(pytz.timezone('US/Eastern')))
    res = dict()
    for vid in response["items"]:

        publish_time = vid["snippet"]["publishTime"]
        est_publish_time, formatted_est_publish_time = \
            convert_utc_to_est(utc_timestamp = publish_time)
        
        publish_time_in_window = (est_publish_time >= start_date) and (est_publish_time <= end_date)

        
        live_status = vid["snippet"]["liveBroadcastContent"]
        if live_status == "none":
            live_status = "completed"

        # we don't want videos that not live/upcoming and not in time window of recent weekend
        if live_status == "completed" and not publish_time_in_window:
            # print(live_status, formatted_est_publish_time, start_date, end_date)
            continue


        
        res[vid["id"]["videoId"]] = {
        "publish_time" : vid["snippet"]["publishTime"],
        "est_publish_time": est_publish_time,
        "formatted_est_publish_time": formatted_est_publish_time,
        "live_status": live_status,
        "title" : vid["snippet"]["title"],
        "channel_title" : vid["snippet"]["channelTitle"],
        "description" : vid["snippet"]["description"],
        "image_url": vid["snippet"]["thumbnails"]["high"]["url"],
        "video_url": "https://www.youtube.com/watch?v=" + vid["id"]["videoId"]
        }

    return res


In [204]:
s,e = get_recent_weekend(est_time=datetime.now(pytz.timezone('US/Eastern')))

Saturday, May 06, 2023: 12:00 AM EST Sunday, May 07, 2023: 11:59 PM EST


In [205]:
res1 = get_youtube_videos(query = "mar thoma church english holy communion")
res2 = get_youtube_videos(query = "mar thoma church english holy qurbana")
# res1 takes precedence
res2.update(res1)

Saturday, May 06, 2023: 12:00 AM EST Sunday, May 07, 2023: 11:59 PM EST
Saturday, May 06, 2023: 12:00 AM EST Sunday, May 07, 2023: 11:59 PM EST


In [212]:
len(res2)

8

In [207]:
import pandas as pd
df = pd.DataFrame.from_dict(res2, orient='index')

# reset the index to create a column for the outer keys
df = df.reset_index()
df = df.rename(columns={'index': 'video_id'})
df = df.sort_values(by="publish_time", ascending=False)

In [209]:
df.at[0, "live_status"] = "completed"
df.at[2, "live_status"] = "live"

In [214]:
df

,video_id,publish_time,est_publish_time,formatted_est_publish_time,live_status,title,channel_title,description,image_url,video_url
0,h7w6cW3LsH4,2023-05-06T13:58:18Z,2023-05-06 09:58:18-04:00,"Saturday, May 06, 2023: 09:58 AM EST",completed,FIRST HOLY QURBANA | KALLUVATHUKKAL IMMANUEL M...,DSMC Department of Sacred Music and Communicat...,ANTI-PIRACY WARNING This content is Copyright ...,https://i.ytimg.com/vi/h7w6cW3LsH4/hqdefault_l...,https://www.youtube.com/watch?v=h7w6cW3LsH4
1,C1ojO3xNups,2023-05-06T11:18:42Z,2023-05-06 07:18:42-04:00,"Saturday, May 06, 2023: 07:18 AM EST",upcoming,Holy Qurbana Live | St Thomas Mar Thoma Syrian...,St Thomas Mar Thoma Syrian Church Pattoor,"7th May 2023, 8 AM | @stthomasmarthomasyrianch...",https://i.ytimg.com/vi/C1ojO3xNups/hqdefault_l...,https://www.youtube.com/watch?v=C1ojO3xNups
2,qXogacQK7dQ,2023-05-06T05:08:57Z,2023-05-06 01:08:57-04:00,"Saturday, May 06, 2023: 01:08 AM EST",live,"Holy Qurbana (English) [7:30 AM, 7 May 2023] -...","St. Thomas Mar Thoma Syrian Church, Santacruz",Holy Communion Service led by Rev. Eapen Abrah...,https://i.ytimg.com/vi/qXogacQK7dQ/hqdefault_l...,https://www.youtube.com/watch?v=qXogacQK7dQ
6,7fn3k4jqo24,2023-05-05T14:42:06Z,2023-05-05 10:42:06-04:00,"Friday, May 05, 2023: 10:42 AM EST",upcoming,Secunderabad-Hyderabad Mar Thoma Church Holy ...,Secunderabad-Hyderabad Mar Thoma Church,Subscribe to our channel for more videos! http...,https://i.ytimg.com/vi/7fn3k4jqo24/hqdefault_l...,https://www.youtube.com/watch?v=7fn3k4jqo24
3,Ct3Sy8A-YJ4,2023-05-05T03:55:38Z,2023-05-04 23:55:38-04:00,"Thursday, May 04, 2023: 11:55 PM EST",upcoming,"7th May, 2023 - Sunday Holy Qurbana | 8.30 A...",SHARON MAR THOMA CHURCH PALARIVATTOM,"Holy Qurbana Live stream from SMTC, Palarivatt...",https://i.ytimg.com/vi/Ct3Sy8A-YJ4/hqdefault_l...,https://www.youtube.com/watch?v=Ct3Sy8A-YJ4
7,8XkmR4AwLgI,2023-04-30T22:07:22Z,2023-04-30 18:07:22-04:00,"Sunday, April 30, 2023: 06:07 PM EST",upcoming,Holy Communion - English - 05/07/2023,Chicago Mar Thoma Church Audio Visual,"Chicago Mar Thoma Church, Des Plaines, IL USA ...",https://i.ytimg.com/vi/8XkmR4AwLgI/hqdefault_l...,https://www.youtube.com/watch?v=8XkmR4AwLgI
4,FeBI0MtTZ0Q,2023-04-23T16:53:46Z,2023-04-23 12:53:46-04:00,"Sunday, April 23, 2023: 12:53 PM EST",upcoming,English Holy Qurbana Service | May 21 @ 10:30 ...,"Immanuel Mar Thoma Church, Houston TX",English Holy Qurbana Service | May 21 @ 10:30 ...,https://i.ytimg.com/vi/FeBI0MtTZ0Q/hqdefault_l...,https://www.youtube.com/watch?v=FeBI0MtTZ0Q
5,Sm1KnSnaKLI,2023-04-23T16:51:21Z,2023-04-23 12:51:21-04:00,"Sunday, April 23, 2023: 12:51 PM EST",upcoming,Malayalam Holy Qurbana Service | May 14 @ 10:3...,"Immanuel Mar Thoma Church, Houston TX",Malayalam Holy Qurbana Service | May 14 @ 10:3...,https://i.ytimg.com/vi/Sm1KnSnaKLI/hqdefault_l...,https://www.youtube.com/watch?v=Sm1KnSnaKLI


In [211]:
df.to_csv("gs://gcf-sources-134756275535-us-central1/church/videos.csv", index=False)



In [157]:
church_locations.to_csv("gs://gcf-sources-134756275535-us-central1/church/locations.csv", index=False)


In [ ]:
# church_locations = pd.DataFrame([["Chicago MTC",42.049530,-87.860060],
#                                  ["Kakkanad", 25.982700, 92.782320],
#                                  ["Trinity Mar Thoma Church Houston", ]
#                                  ], columns = ["church","latitude","longitude"])

In [183]:
videos_df = pd.read_csv("gs://gcf-sources-134756275535-us-central1/church/videos.csv")
videos_df

,video_id,publish_time,est_publish_time,formatted_est_publish_time,live_status,title,channel_title,description,image_url
0,bxZGumzhEEw,2023-05-01T03:29:20Z,2023-04-30 23:29:20-04:00,"Sunday, April 30, 2023: 11:29 PM EST",neither live nor upcoming,"English Divine Service | April 30th, 2023 | Tr...",Trinity Mar Thoma Church Houston,"English Divine Service | April 30th, 2023 | Tr...",https://i.ytimg.com/vi/bxZGumzhEEw/hqdefault.jpg
1,8XkmR4AwLgI,2023-04-30T22:07:22Z,2023-04-30 18:07:22-04:00,"Sunday, April 30, 2023: 06:07 PM EST",upcoming,Holy Communion - English - 05/07/2023,Chicago Mar Thoma Church Audio Visual,"Chicago Mar Thoma Church, Des Plaines, IL USA ...",https://i.ytimg.com/vi/8XkmR4AwLgI/hqdefault_l...
2,xMUwY1AHpKs,2023-04-30T20:39:36Z,2023-04-30 16:39:36-04:00,"Sunday, April 30, 2023: 04:39 PM EST",neither live nor upcoming,"English Holy Communion Service - April 30, 2023",The Canadian Mar Thoma Church,"""Don't worry about anything; instead pray abou...",https://i.ytimg.com/vi/xMUwY1AHpKs/hqdefault.jpg
3,P5h0jyRbn-o,2023-04-30T18:17:46Z,2023-04-30 14:17:46-04:00,"Sunday, April 30, 2023: 02:17 PM EST",neither live nor upcoming,"The Mar Thoma Church of Dallas, Carrollton | E...",MTCD Carrollton,NaN,https://i.ytimg.com/vi/P5h0jyRbn-o/hqdefault.jpg
4,fCE63lynLw8,2023-04-30T18:04:28Z,2023-04-30 14:04:28-04:00,"Sunday, April 30, 2023: 02:04 PM EST",neither live nor upcoming,Secunderabad-Hyderabad Mar Thoma Church Holy ...,Secunderabad-Hyderabad Mar Thoma Church,Subscribe to our channel for more videos! http...,https://i.ytimg.com/vi/fCE63lynLw8/hqdefault.jpg
5,7oRRnoWqym4,2023-04-30T17:01:01Z,2023-04-30 13:01:01-04:00,"Sunday, April 30, 2023: 01:01 PM EST",neither live nor upcoming,"Trinity Marthoma Church TVM, 30-04-2023 Sunday...",Trinity Mar Thoma Church Trivandrum,"Trinity Marthoma Church TVM, Sunday Service @8AM.",https://i.ytimg.com/vi/7oRRnoWqym4/hqdefault.jpg
6,SlOIv-1FdtM,2023-04-30T16:59:12Z,2023-04-30 12:59:12-04:00,"Sunday, April 30, 2023: 12:59 PM EST",neither live nor upcoming,Holy Qurbana(English Service) Live | St Thomas...,St Thomas Mar Thoma Syrian Church Pattoor,"30th April 2023, 8 AM | @stthomasmarthomasyria...",https://i.ytimg.com/vi/SlOIv-1FdtM/hqdefault.jpg
7,JLznm0YVwAY,2023-04-30T16:57:28Z,2023-04-30 12:57:28-04:00,"Sunday, April 30, 2023: 12:57 PM EST",neither live nor upcoming,Mar Thoma Holy Qurbana LIVE - Rev. Manoj Idicu...,St Paul's Marthoma Church,Mar Thoma Holy Qurbana LIVE - Rev. Manoj Idicu...,https://i.ytimg.com/vi/JLznm0YVwAY/hqdefault.jpg
8,AIWD84B-oZ4,2023-04-30T16:55:08Z,2023-04-30 12:55:08-04:00,"Sunday, April 30, 2023: 12:55 PM EST",neither live nor upcoming,HOLY COMMUNION SERVICE 30-04-2023 @ 8 AM | JE...,Jerusalem Mar Thoma Church Nanthencode,Live service from Jerusalem Mar Thoma Church N...,https://i.ytimg.com/vi/AIWD84B-oZ4/hqdefault.jpg
9,s8Pz0ujWGNg,2023-04-30T16:23:52Z,2023-04-30 12:23:52-04:00,"Sunday, April 30, 2023: 12:23 PM EST",neither live nor upcoming,"HOLY COMMUNION | EBENEZER MAR THOMA CHURCH, PE...","Ebenezer MarThoma Church, Peroorkada",NaN,https://i.ytimg.com/vi/s8Pz0ujWGNg/hqdefault.jpg


In [185]:
videos_df.query("live_status == 'live'")

,video_id,publish_time,est_publish_time,formatted_est_publish_time,live_status,title,channel_title,description,image_url
8,AIWD84B-oZ4,2023-04-30T16:55:08Z,2023-04-30 12:55:08-04:00,"Sunday, April 30, 2023: 12:55 PM EST",live,HOLY COMMUNION SERVICE 30-04-2023 @ 8 AM | JE...,Jerusalem Mar Thoma Church Nanthencode,Live service from Jerusalem Mar Thoma Church N...,https://i.ytimg.com/vi/AIWD84B-oZ4/hqdefault.jpg


In [154]:
result_df

,church,latitude,longitude
0,Trinity Mar Thoma Church Houston,29.61976,-95.31938
1,Chicago Mar Thoma Church Audio Visual,42.04898,-87.86070
2,The Canadian Mar Thoma Church,43.96229,-79.26883
3,MTCD Carrollton,32.99775,-96.92082
4,Secunderabad-Hyderabad Mar Thoma Church,17.44920,78.50798
5,Trinity Mar Thoma Church Trivandrum,8.52206,76.93051
6,St Thomas Mar Thoma Syrian Church Pattoor,8.49662,76.93731
7,St Paul's Marthoma Church,9.60441,76.80321
8,Jerusalem Mar Thoma Church Nanthencode,8.51786,76.95650
9,"Ebenezer MarThoma Church, Peroorkada",8.54100,76.97043


last_updated = datetime.now(pytz.timezone('US/Eastern'))

In [169]:
est_time = datetime.now(pytz.timezone('US/Eastern'))

formatted_est_time = est_time.strftime('%A, %B %d, %Y: %I:%M %p EST')
print(formatted_est_time)

Saturday, May 06, 2023: 11:59 AM EST


In [175]:
pd.DataFrame([formatted_est_time], columns = ["update_date"])

,update_date
0,"Saturday, May 06, 2023: 11:59 AM EST"


In [176]:
pd.DataFrame([formatted_est_time], columns = ["update_date"]).to_csv("gs://gcf-sources-134756275535-us-central1/church/update_date.csv", index=False)


In [181]:
update_date = pd.read_csv("gs://gcf-sources-134756275535-us-central1/church/update_date.csv").iloc[0,0]

In [182]:
update_date

'Saturday, May 06, 2023: 11:59 AM EST'